In [1]:
def ldata(archive):
    f=open(archive)
    data=[]
    for line in f:
        line=line.strip()
        col=line.split()
        data.append(col)
    return data
def ldata2(archive):
    f=open(archive)
    data=[]
    for line in f:
        line=line.strip()
        col=line.split()
        data.append([col[0],col[1]])
    return data


In [2]:


import numpy as np
import networkx as nx
#import matplotlib.pyplot as plt
import pandas as pd

red_proteinas=ldata("tc02Data/yeast_AP-MS.txt")
red_binarias=ldata("tc02Data/yeast_Y2H.txt")
red_literatura=ldata("tc02Data/yeast_LIT.txt")
red_esencialHe=ldata("tc02Data/Essential_ORFs_paperHe.txt")
red_litreguly=ldata2("tc02Data/yeast_LIT_Reguly.txt")



In [3]:
G = nx.Graph()#directed
G.add_edges_from(red_proteinas)
G1 = nx.Graph()#directed
G1.add_edges_from(red_binarias)
G2 = nx.Graph()#directed
G2.add_edges_from(red_literatura)
G3 = nx.Graph()#directed
G3.add_edges_from(red_litreguly)

In [4]:
def K(H):
    k=sum(H.degree(k) for k in H)/H.number_of_nodes()
    salida=[k]
    return salida

In [5]:
data = pd.DataFrame({"Red":["Proteinas","Binarias","Literatura", "Literatura Reg."],
                     "Número de Nodos":[G.number_of_nodes(),G1.number_of_nodes(),G2.number_of_nodes(), G3.number_of_nodes()],
                     "Número de Enlaces":[G.number_of_edges(),G1.number_of_edges(),G2.number_of_edges(),G3.number_of_edges()],
                 "Grado medio ":[K(G),K(G1),K(G2),K(G3)],
                 "Clustering promedio":[nx.average_clustering(G),nx.average_clustering(G1),nx.average_clustering(G2),nx.average_clustering(G3)] })
cols= ['Red', 'Número de Nodos', 'Número de Enlaces', 'Grado medio ','Clustering promedio']

data[cols]


,Red,Número de Nodos,Número de Enlaces,Grado medio,Clustering promedio
0,Proteinas,1622,9070,[11.183723797780518],0.554636
1,Binarias,2018,2930,[2.9038652130822595],0.046194
2,Literatura,1536,2925,[3.80859375],0.292492
3,Literatura Reg.,3309,11859,[7.167724388032639],0.260976
